In [6]:
# %% === SCANNER DE TENDÊNCIA (ADX + MÉDIAS MÓVEIS POR ÍNDICE) ===
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import pytz
from pathlib import Path
from IPython.display import display

# ===============================================================
# 🧾 ÍNDICES DISPONÍVEIS (baseados na pasta /tickers):
# IBOV, IBRA, IBRX, IBXL, IC02, ICON, IDIV, IEEX, IFNC,
# IGCT, IGCX, IGNM, IMAT, IMOB, INDX, ISEE, ITAG, IVBX,
# MLCX, SMLL, UTIL
# ===============================================================

# 🧩 Escolha o índice e o caminho relativo à pasta do projeto
INDICE = "IBOV"  # exemplo: "IBOV", "IBRA", "SMLL", etc.
BASE_PATH = Path.cwd() / "TICKERS"  # caminho relativo à pasta /tickers

# ===============================================================
# ⚙️ CONFIGURAÇÕES GERAIS
# ===============================================================
CONFIG = {
    "timeframe": mt5.TIMEFRAME_H4,      # Exemplo: mt5.TIMEFRAME_M15, mt5.TIMEFRAME_H1, mt5.TIMEFRAME_D1
    "history_days": 100,                # Histórico de dias
    "adx_period": 14,                   # Período do ADX
    "adx_min": 35,                      # ADX mínimo para tendência
    "sma_periods": [9, 21, 50],         # Médias móveis a usar
    "display_decimals": 2,              # Casas decimais
}

# ===============================================================
# 🧠 FUNÇÕES AUXILIARES
# ===============================================================

def initialize_mt5():
    """Inicializa a conexão com o MetaTrader 5."""
    if not mt5.initialize():
        raise RuntimeError(f"Erro ao inicializar o MT5: {mt5.last_error()}")
    print("✅ Conexão com o MetaTrader 5 estabelecida.")

def calculate_adx(df, window=14):
    """Calcula o ADX a partir das colunas OHLC."""
    df['plus_dm'] = df['high'].diff().clip(lower=0)
    df['minus_dm'] = df['low'].diff().clip(upper=0).abs()
    df['tr'] = pd.concat([
        df['high'] - df['low'],
        abs(df['high'] - df['close'].shift(1)),
        abs(df['low'] - df['close'].shift(1))
    ], axis=1).max(axis=1)
    df['plus_di'] = 100 * (df['plus_dm'] / df['tr']).rolling(window=window).mean()
    df['minus_di'] = 100 * (df['minus_dm'] / df['tr']).rolling(window=window).mean()
    df['dx'] = (abs(df['plus_di'] - df['minus_di']) / (df['plus_di'] + df['minus_di'])) * 100
    return df['dx'].rolling(window=window).mean()

def get_mt5_data(ticker, timeframe, days):
    """Baixa dados históricos do MT5 para o ticker informado."""
    tz = pytz.timezone('America/Sao_Paulo')
    agora = datetime.now(tz)
    inicio = (agora - timedelta(days=days)).astimezone(pytz.utc)
    fim = agora.astimezone(pytz.utc)
    rates = mt5.copy_rates_range(ticker, timeframe, inicio, fim)
    if rates is None or len(rates) == 0:
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

def analyze_ticker(ticker, cfg):
    """Verifica se o ticker atende às condições de tendência."""
    df = get_mt5_data(ticker, cfg["timeframe"], cfg["history_days"])
    if df is None:
        return None

    # Calcula as médias móveis dinamicamente
    for p in cfg["sma_periods"]:
        df[f'SMA_{p}'] = df['close'].rolling(window=p).mean()

    # Calcula o ADX
    df['ADX'] = calculate_adx(df, window=cfg["adx_period"])
    last = df.iloc[-1]

    # Filtros
    if pd.isna(last['ADX']) or last['ADX'] < cfg["adx_min"]:
        return None
    if not all(last['close'] > last[f'SMA_{p}'] for p in cfg["sma_periods"]):
        return None

    # Cria o dicionário dinamicamente com base nas médias definidas
    resultado = {
        'Ticker': ticker,
        'Preço Atual': round(last['close'], cfg["display_decimals"]),
        **{f'Média {p}': round(last[f'SMA_{p}'], cfg["display_decimals"]) for p in cfg["sma_periods"]},
        'ADX Atual': round(last['ADX'], cfg["display_decimals"])
    }

    return resultado

# ===============================================================
# 🚀 EXECUÇÃO PRINCIPAL
# ===============================================================
if __name__ == "__main__":
    initialize_mt5()

    excel_path = BASE_PATH / f"tickers_{INDICE}.xlsx"

    if not excel_path.exists():
        mt5.shutdown()
        raise FileNotFoundError(f"❌ Arquivo não encontrado: {excel_path}\n"
                                "Verifique se o arquivo do índice está na pasta 'tickers/'.")

    df_tickers = pd.read_excel(excel_path)
    tickers = df_tickers['Ticker'].dropna().tolist()

    results = []
    for ticker in tickers:
        result = analyze_ticker(ticker, CONFIG)
        if result:
            results.append(result)

    # Exibe resultados
    if results:
        results_df = pd.DataFrame(results).sort_values(by='ADX Atual', ascending=False)
        pd.set_option('display.float_format', lambda x: f"{x:.{CONFIG['display_decimals']}f}")
        pd.set_option('display.max_rows', None)

        # Nome legível do timeframe
        timeframe_names = {
            mt5.TIMEFRAME_M1: "1 Minuto",
            mt5.TIMEFRAME_M5: "5 Minutos",
            mt5.TIMEFRAME_M15: "15 Minutos",
            mt5.TIMEFRAME_M30: "30 Minutos",
            mt5.TIMEFRAME_H1: "1 Hora",
            mt5.TIMEFRAME_H4: "4 Horas",
            mt5.TIMEFRAME_D1: "Diário"
        }
        tf_str = timeframe_names.get(CONFIG["timeframe"], str(CONFIG["timeframe"]))

        print(f"\n📊 Resultados para o índice {INDICE}:")
        print(f"Médias utilizadas: {CONFIG['sma_periods']} períodos | Timeframe: {tf_str}\n")
        display(results_df)
    else:
        print(f"⚠️ Nenhum ativo do índice {INDICE} acima das médias {CONFIG['sma_periods']} e com ADX > {CONFIG['adx_min']}.")

    mt5.shutdown()


✅ Conexão com o MetaTrader 5 estabelecida.

📊 Resultados para o índice IBOV:
Médias utilizadas: [9, 21, 50] períodos | Timeframe: 4 Horas



,Ticker,Preço Atual,Média 9,Média 21,Média 50,ADX Atual
22,TAEE11,40.56,40.32,39.44,38.06,73.92
24,CYRE3,33.54,32.59,31.49,30.50,72.57
20,EGIE3,42.80,42.69,41.46,40.44,66.88
6,ABEV3,13.21,13.17,13.04,12.55,64.09
10,EQTL3,37.93,37.55,36.77,35.99,62.36
14,SBSP3,138.11,136.77,134.73,132.67,60.69
11,PRIO3,38.87,38.51,37.33,36.68,58.66
18,ALOS3,26.20,26.08,25.53,24.86,56.67
3,ELET3,58.68,58.08,56.90,55.43,56.55
19,ELET6,61.94,61.33,60.28,58.67,56.27
